In [3]:
import torch
import cv2
import numpy as np

# Load YOLOv5 model (from ultralytics/yolov5 repo)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Set model to eval mode
model.eval()

cap = cv2.VideoCapture("C:\\Users\\marip\\Downloads\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mov")  # or replace 0 with a video file path

# Class indices for YOLOv5
PEDESTRIAN_CLASS = 'person'
TRAFFIC_LIGHT_CLASS = 'traffic light'

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run inference
    results = model(img)

    # Parse results
    detections = results.pandas().xyxy[0]  # Bounding boxes with confidence

    for _, row in detections.iterrows():
        class_name = row['name']
        conf = row['confidence']
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])

        if conf > 0.5:
            if class_name == PEDESTRIAN_CLASS:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Pedestrian {conf:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            elif class_name == TRAFFIC_LIGHT_CLASS:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, f'Traffic Light {conf:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Display result
    cv2.imshow("YOLOv5 Pedestrian & Traffic Light Detection", frame)

    # Exit key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\marip/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-6-3 Python-3.9.19 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\marip/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\marip/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\marip/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\marip/.cache\torch\hub\ultralytics_yolov5_mas

In [10]:
import cv2
import numpy as np

# Load Haar cascade for full body detection
pedestrian_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')

# Start video capture (0 for webcam)
cap = cv2.VideoCapture("C:\\Users\\marip\\Downloads\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mov")

# Define region of interest (ROI) as the crosswalk zone
# You can adjust these values to match your video scene
crosswalk_top_left = (150, 300)
crosswalk_bottom_right = (500, 450)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize and convert to grayscale
    frame = cv2.resize(frame, (640, 480))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect pedestrians
    pedestrians = pedestrian_cascade.detectMultiScale(gray, 1.1, 4)

    # Draw crosswalk area
    cv2.rectangle(frame, crosswalk_top_left, crosswalk_bottom_right, (255, 255, 0), 2)
    alert_triggered = False

    for (x, y, w, h) in pedestrians:
        # Draw pedestrian bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Check if pedestrian is within the crosswalk region
        if (crosswalk_top_left[0] < x < crosswalk_bottom_right[0] and
                crosswalk_top_left[1] < y + h < crosswalk_bottom_right[1]):
            alert_triggered = True

    # Show alert if pedestrian in crosswalk
    if alert_triggered:
        cv2.putText(frame, "ALERT: Crosswalk Occupied!", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        # Optional: play sound alert with playsound or winsound

    # Display frame
    cv2.imshow('Pedestrian Detection with Crosswalk Alert', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
